In [1]:
import pandas as pd
books=pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline
classifier=pipeline("text-classification",model="j-hartmann/emotion-english-distilroberta-base",top_k=None,device="cuda")
classifier("I love this book")

Device set to use cuda


[[{'label': 'joy', 'score': 0.9862679243087769},
  {'label': 'surprise', 'score': 0.005481821019202471},
  {'label': 'sadness', 'score': 0.003376911161467433},
  {'label': 'neutral', 'score': 0.002553065773099661},
  {'label': 'anger', 'score': 0.0012129773385822773},
  {'label': 'disgust', 'score': 0.0006926582427695394},
  {'label': 'fear', 'score': 0.0004146655846852809}]]

In [3]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [4]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.14038559794425964},
  {'label': 'fear', 'score': 0.06816219538450241},
  {'label': 'joy', 'score': 0.04794243723154068},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.0026284728664904833},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937053322792053},
  {'label': 'disgust', 'score': 0.2735919654369354},
  {'label': 'joy', 'score': 0.10908277332782745},
  {'label': 'sadness', 'score': 0.09362724423408508},
  {'label': 'anger', 'score': 0.04047832265496254},
  {'label': 'surprise', 'score': 0.02697017788887024},
  {'label': 'fear', 'score': 0.006879044696688652}],
 [{'label': 'neutral', 'score': 0.6462171077728271},
  {'label': 'sadness', 'score': 0.24273230135440826},
  {'label': 'disgust', 'score': 0.04342260956764221},
  {'label': 'surprise', 'score': 0.028300557285547256},
  {'label': 'joy', 'score': 0.01421146

In [5]:
sentences=books["description"][0].split(".")
predictions=classifier(sentences)

In [6]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [7]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.14038559794425964},
 {'label': 'fear', 'score': 0.06816219538450241},
 {'label': 'joy', 'score': 0.04794243723154068},
 {'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [8]:
sorted(predictions[0],key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'fear', 'score': 0.06816219538450241},
 {'label': 'joy', 'score': 0.04794243723154068},
 {'label': 'neutral', 'score': 0.14038559794425964},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296027541160583}]

In [9]:
import numpy as np
emotion_labels=["anger","disgust","fear","joy","neutral","sadness","surprise"] 
isbn=[]
emotion_scores={label:[] for label in emotion_labels}

def calculate_max_emotion_score(predictions):
    per_emotion_score={label:[] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction=sorted(prediction,key=lambda x:x["label"])
        for index,label in enumerate(emotion_labels):
            per_emotion_score[label].append(sorted_prediction[index]["score"])
    return {label:np.max(scores) for label,scores in per_emotion_score.items()}

In [10]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences=books["description"][i].split(".")
    predictions=classifier(sentences)
    max_score=calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_score[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [11]:
emotion_scores

{'anger': [0.06413362920284271,
  0.6126183867454529,
  0.06413362920284271,
  0.3514839708805084,
  0.08141238242387772,
  0.2322247326374054,
  0.5381839275360107,
  0.06413362920284271,
  0.3006700873374939,
  0.06413362920284271],
 'disgust': [0.2735919654369354,
  0.3482854962348938,
  0.10400665551424026,
  0.1507224291563034,
  0.18449531495571136,
  0.7271749973297119,
  0.15585508942604065,
  0.10400665551424026,
  0.2794812321662903,
  0.17792671918869019],
 'fear': [0.9281679391860962,
  0.9425276517868042,
  0.9723208546638489,
  0.36070650815963745,
  0.0950433611869812,
  0.0513627864420414,
  0.7474288940429688,
  0.4044960141181946,
  0.9155242443084717,
  0.0513627864420414],
 'joy': [0.9327974319458008,
  0.7044206857681274,
  0.76723712682724,
  0.2518812119960785,
  0.0405643954873085,
  0.043375879526138306,
  0.8725649118423462,
  0.0405643954873085,
  0.0405643954873085,
  0.0405643954873085],
 'neutral': [0.6462171077728271,
  0.8879395127296448,
  0.54947686195

In [12]:
from tqdm import tqdm
emotion_labels=["anger","disgust","fear","joy","neutral","sadness","surprise"] 
isbn=[]
emotion_scores={label:[] for label in emotion_labels}
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences=books["description"][i].split(".")
    predictions=classifier(sentences)
    max_score=calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_score[label])

100%|██████████| 5197/5197 [01:24<00:00, 61.26it/s] 


In [13]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn

In [14]:
emotions_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273592,0.928168,0.932797,0.646217,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934


In [15]:
books=pd.merge(books,emotions_df,on="isbn13")

In [16]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883: A NOVEL THAT READERS and critic...,Fiction,0.064134,0.273592,0.928168,0.932797,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982: A new 'Christie for Christmas' ...,Fiction,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736: A memorable, mesmerizing heroin...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897: Lewis' work on the nature of lo...,Nonfiction,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934: ""In The Problem of Pain, C.S. L...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475880,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222: On A Train Journey Home To Nort...,Fiction,0.148209,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014: This book tells the tale of a m...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883199,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623: Wisdom to Create a Life of Pass...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535: This collection of the timeless...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078765


In [17]:
books.to_csv("books_with_emotions.csv",index=False)